## LEOHS Apply Harmonization Tool
This script applies harmonization equations to either multiband Landsat images or a directory containing Landsat images.

In [1]:
import os,re
import pandas as pd
import rasterio #Please note the image writing settings 
import numpy as np

In [2]:
Landsat_image_or_dir=r'E:\GIS\Landsat Normalization\GLAD\L7_turkey.tif'
#can be either an image path or dir path. Images need to be 6 band rasters.
input_harmonization="""
B: LS8 = 0.9347LS7 - 0.0094
G: LS8 = 0.9305LS7 - 0.0048
R: LS8 = 0.9301LS7 - 0.0055
NIR: LS8 = 0.9685LS7 + 0.0016
SWIR1: LS8 = 0.9460LS7 - 0.0005
SWIR2: LS8 = 0.9284LS7 + 0.0032

""" #copy equations from textfile into a raw string like this

In [3]:
def detect_conversion_direction(text_block):
    first_line = text_block.strip().splitlines()[0]
    if "LS8 =" in first_line:
        return "2L8"
    elif "LS7 =" in first_line:
        return "2L7"
    else:
        return "Unknown"
def parse_harmonization_text(text_block):
    data = []
    for line in text_block.strip().splitlines():
        parts = line.strip().split(': ')
        if len(parts) < 2 or '=' not in parts[1]:
            continue  # Skip malformed lines
        band = parts[0]
        equation = parts[1]
        _, right = equation.split(' = ')
        match = re.match(r'([+-]?\d*\.?\d+)([A-Za-z0-9_]+)\s*([+-])\s*(\d*\.?\d+)', right)
        if match:
            slope = float(match.group(1))
            sign = 1 if match.group(3) == '+' else -1
            intercept = sign * float(match.group(4))
            data.append([band, slope, intercept])
    df = pd.DataFrame(data, columns=['Band', 'Slope', 'Intercept'])
    return df
def apply_harmonization(input_harmonization, input_path):
    if os.path.isdir(input_path):
        images = [os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith(".tif")]
        print("Found .tif files:", [os.path.basename(f) for f in images])
    elif os.path.isfile(input_path) and input_path.endswith(".tif"):
        images = [input_path]
    else:
        print("Invalid input path")
        return
    conversion_prefix = detect_conversion_direction(input_harmonization)
    print(f'Converting images {conversion_prefix}')
    df = parse_harmonization_text(input_harmonization)
    print(df)

    for landsat_image in images:
        filename = os.path.basename(landsat_image)
        print(f"Processing {filename}")
        with rasterio.open(landsat_image) as src:
            meta = src.meta.copy()
            bands = src.count
            nodata_value = meta.get('nodata')
            output_data = np.zeros((bands, meta['height'], meta['width']), dtype=np.float32)
            for i in range(bands):
                band_name = df.iloc[i]['Band']
                slope = df.iloc[i]['Slope']
                intercept = df.iloc[i]['Intercept']
                band_data = src.read(i + 1).astype(np.float32)
                # Scale if max pixel value > 1000
                max_pixel = np.nanmax(band_data)
                if max_pixel > 1000:
                    print(f"Scaling band {band_name}")
                    meta.update(dtype='float32')
                    band_data = band_data * 0.0000275 - 0.2
                if nodata_value is not None:
                    mask = band_data == nodata_value
                    band_data = np.where(mask, np.nan, band_data * slope + intercept)
                    band_data = np.where(np.isnan(band_data), nodata_value, band_data)
                else:
                    band_data = band_data * slope + intercept
                band_data = np.where((band_data < 0) | (band_data > 1), -1, band_data)
                output_data[i] = band_data
                print(f"Processed band {band_name} and preserved no data pixels")
            output_filename = os.path.join(os.path.dirname(landsat_image), f"{conversion_prefix}_{filename}")
            ###Note the image writing settings###
            meta.update(nodata = -1,compress='zstd',predictor=2,
                tiled=True,blockxsize=512,blockysize=512,NUM_THREADS='ALL_CPUS',bigtiff='YES')
            with rasterio.open(output_filename, 'w', **meta) as dst:
                for i in range(bands):
                    dst.write(output_data[i], i + 1)

            print(f"Saved harmonized image as {output_filename}")

In [4]:
apply_harmonization(input_harmonization, Landsat_image_or_dir)

Converting images 2L8
    Band   Slope  Intercept
0      B  0.9347    -0.0094
1      G  0.9305    -0.0048
2      R  0.9301    -0.0055
3    NIR  0.9685     0.0016
4  SWIR1  0.9460    -0.0005
5  SWIR2  0.9284     0.0032
Processing L7_turkey.tif
Scaling band B
Processed band B and preserved no data pixels
Scaling band G
Processed band G and preserved no data pixels
Scaling band R
Processed band R and preserved no data pixels
Scaling band NIR
Processed band NIR and preserved no data pixels
Scaling band SWIR1
Processed band SWIR1 and preserved no data pixels
Scaling band SWIR2
Processed band SWIR2 and preserved no data pixels
Saved harmonized image as E:\GIS\Landsat Normalization\GLAD\2L8_L7_turkey.tif


Saved unscaled reflectance image as E:\GIS\Landsat Normalization\GLAD\GLAD_turkey_reflectance.tif
